Contents of `mlprimitives.meta.py`

```python
import pandas as pd
from mlblocks import MLBlock


class ApplyPrimitive:

    def __init__(self, primitive_name, keyword='x', axis=0, init_params=None, **hyperparameters):
        self.keyword = keyword
        if init_params:
            init_params = init_params.copy()
            init_params.update(hyperparameters)
        else:
            init_params = hyperparameters

        self.mlblock = MLBlock(primitive_name, **init_params)
        self.axis = axis

    def apply(self, x):
        kwargs = {
            self.keyword: x
        }
        return self.mlblock.produce(**kwargs)

    def produce(self, X):
        dataframe = isinstance(X, pd.DataFrame)
        if not dataframe:
            X = pd.DataFrame(X)

        X = X.apply(self.apply, axis=self.axis)

        if dataframe:
            return X
        else:
            return X.values
```

Contents of `mlprimitives/primitives/mlprimitives.meta.ApplyPrimitive.json`

```json
{
    "name": "mlprimitives.meta.ApplyPrimitive",
    "primitive": "mlprimitives.meta.ApplyPrimitive",
    "produce": {
        "method": "produce",
        "args": [
            {
                "name": "X",
                "type": "ndarray"
            }
        ],
        "output": [
            {
                "name": "X",
                "type": "ndarray"
            }
        ]
    },
    "hyperparameters": {
        "fixed": {
            "primitive_name": {
                "type": "str",
                "description": "primitive to apply"
            },
            "keyword": {
                "type": "str",
                "default": "x",
                "description": "names of the argument expected by the primitive"
            },
            "axis": {
                "type": "int",
                "default": 0,
                "description": "Axis to which the primitive will be applied"
            },
            "init_params": {
                "type": "dict",
                "default": {},
                "description": "primitive init_params"
            }
        }
    }
}
```

Contents of `mlprimitives.test.py`

```python
def add(x, k):
    return x.sum() + k
```

Contents of `mlprimitives/primitives/mlprimitives.test.add.json`

```json
{
    "name": "mlprimitives.test.add",
    "primitive": "mlprimitives.test.add",
    "produce": {
        "args": [
            {
                "name": "x"
            }
        ],
        "output": [
            {
                "name": "x"
            }
        ]
    },
    "hyperparameters": {
        "fixed": {
            "k": {
                "type": "int"
            }
        }
    }
}
```

In [1]:
import pandas as pd

X = pd.DataFrame([
    [1, 2],
    [3, 4]
])

In [2]:
X

,0,1
0,1,2
1,3,4


In [3]:
from mlblocks import MLBlock, MLPipeline

In [4]:
block = MLBlock('mlprimitives.test.add', k=10)

In [5]:
block.produce(x=X[0])

14

In [20]:
block = MLBlock('mlprimitives.test.sum', k=10)

In [21]:
block.produce(x=X, y=1000, z=100)

0    1114
1    1116
dtype: int64

In [22]:
meta = MLBlock(
    'mlprimitives.meta.ApplyPrimitive',
    primitive_name='mlprimitives.test.add',
    init_params={'k': 10},
)

In [23]:
meta.produce(X=X)

,0
0,13
1,17


In [24]:
meta = MLBlock(
    'mlprimitives.test.add.by_row',
    k=10
)

In [25]:
meta.produce(X=X)

,0
0,13
1,17


In [26]:
meta = MLBlock(
    'mlprimitives.test.sum.by_row',
    k=10
)

In [27]:
meta.produce(X=X, Y=[1000, 2000], z=100)

,0
0,1113
1,2117


In [31]:
meta.set_hyperparameters({'k': 20})

In [32]:
meta.produce(X=X, Y=[1000, 2000], z=100)

,0
0,1123
1,2127


In [33]:
meta.get_hyperparameters()

{'primitive_name': 'mlprimitives.test.sum',
 'keywords': {'X': 'x', 'Y': 'y'},
 'axis': 0,
 'zip': ['Y'],
 'k': 20,
 'init_params': {'k': 20}}

In [35]:
pipeline_dict = {
    'primitives': [
        'mlprimitives.meta.ApplyPrimitive',
    ],
    'init_params': {
        'mlprimitives.meta.ApplyPrimitive#1': {
            'primitive_name': 'mlprimitives.test.add',
            'axis': 1,
            'init_params': {
                'k': 10
            }
        }
    }
}

pipeline = MLPipeline(pipeline_dict)

In [36]:
pipeline.get_tunable_hyperparameters()

{'mlprimitives.meta.ApplyPrimitive#1': {}}

In [37]:
pipeline.get_hyperparameters()

{'mlprimitives.meta.ApplyPrimitive#1': {'primitive_name': 'mlprimitives.test.add',
  'keywords': {'X': 'x'},
  'axis': 1,
  'init_params': {'k': 10},
  'zip': []}}

In [38]:
pipeline.fit(X=X)

In [39]:
pipeline.predict(X=X)

,0
0,14
1,16


In [40]:
pipeline_dict = {
    'primitives': [
        'mlprimitives.meta.ApplyPrimitive',
    ],
    'init_params': {
        'mlprimitives.meta.ApplyPrimitive#1': {
            'primitive_name': 'mlprimitives.test.add',
            'axis': 1,
            'init_params': {
                'k': 10
            }
        }
    },
    'tunable_hyperparameters': {
        'mlprimitives.meta.ApplyPrimitive#1': {
            'k': {
                'type': 'int'
            }
        }
    }
}

pipeline = MLPipeline(pipeline_dict)

In [41]:
pipeline.get_tunable_hyperparameters()

{'mlprimitives.meta.ApplyPrimitive#1': {'k': {'type': 'int'}}}

In [42]:
meta = MLBlock(
    'mlprimitives.test.add.by_row',
    k=10
)

In [43]:
meta.get_tunable_hyperparameters()

{}